In [1]:
import xarray as xr
from datetime import datetime as dt
import numpy as np
import pandas as pd
from os import listdir, path

In [3]:
date_i_want = dt(2022, 6, 17)
tfm_path = f'/Volumes/LtgSSD/tobac_saves/tobac_Save_{date_i_want.strftime("%Y%m%d")}/Track_features_merges.nc'
tfm = xr.open_dataset(tfm_path)

In [4]:
tfm

<xarray.Dataset> Size: 2GB
Dimensions:                           (feature: 24166, time: 387, y: 1001,
                                       x: 1001, track: 936, cell: 1165)
Coordinates:
  * feature                           (feature) int64 193kB 1 2 ... 24165 24166
  * time                              (time) datetime64[ns] 3kB 2022-06-17T00...
  * y                                 (y) float64 8kB -2.5e+05 ... 2.5e+05
  * x                                 (x) float64 8kB -2.5e+05 ... 2.5e+05
  * track                             (track) float64 7kB 0.0 1.0 ... 935.0
  * cell                              (cell) int64 9kB -1 1 4 ... 8586 8596 8598
Data variables: (12/19)
    feature_time_index                (feature) int64 193kB ...
    feature_id_this_frame             (feature) int64 193kB ...
    feature_hdim1_coordinate          (feature) float64 193kB ...
    feature_hdim2_coordinate          (feature) float64 193kB ...
    feature_grid_cell_count           (feature) int64 193kB ...
    feature_threshold_max             (feature) float64 193kB ...
    ...                                ...
    feature_parent_cell_elapsed_time  (feature) timedelta64[ns] 193kB ...
    segmentation_mask                 (time, y, x) int32 2GB ...
    cell_parent_track_id              (cell) float64 9kB ...
    feature_parent_track_id           (feature) float64 193kB ...
    track_child_cell_count            (track) float64 7kB ...
    cell_child_feature_count          (cell) float64 9kB ...

In [5]:
def check_all(arr_1, arr_2, map):
    arr_1_flat = arr_1.flatten()
    arr_2_flat = arr_2.flatten()
    for i in range(arr_1_flat.shape[0]):
        if np.isnan(arr_1_flat[i]):
            if np.isnan(arr_2_flat[i]):
                continue
            else:
                print(f'Error at index {i}! Expected NaN, got {arr_2_flat[i]}')
        else:
            arr_1_val = arr_1_flat[i]
            arr_2_expected = map[int(arr_1_val) - 1]
            arr_2_actual = arr_2_flat[i]
            if arr_2_expected != arr_2_actual:
                print(f'Error at index {i}! Got {arr_2_expected} from replacement index {int(arr_1_val) + 1}, got {arr_2_actual} from checked array')
                break

In [6]:
radar_top_path = f'/Volumes/LtgSSD/nexrad_zarr/{date_i_want.strftime('%B').upper()}/{date_i_want.strftime('%Y%m%d')}/'
radar_files = sorted(listdir(radar_top_path))

In [7]:
tfm_dts = tfm.time.data.astype('datetime64[s]').astype(dt)

In [8]:
feature_eet = np.full(tfm.feature.data.shape, np.nan)

In [9]:
def find_eet_feature(tfm, radar_path, time_idx):
        radar = xr.open_dataset(radar_path, engine='zarr')
        feature_indicies_at_time = np.nonzero(tfm.feature_time_index.data == time_idx)[0]
        features_at_time = tfm.isel(feature=feature_indicies_at_time).isel(time=time_idx)
        feature_eet = np.full(features_at_time.feature.data.shape, np.nan)
        for j, feat_to_find in enumerate(features_at_time.feature.data):
            this_seg_mask = features_at_time.segmentation_mask.data
            this_eet = radar.eet_sam.isel(time=0).data
            if not np.any(this_seg_mask == feat_to_find):
                continue
            feature_eet[j] = np.nanmax(this_eet[this_seg_mask == feat_to_find])
        start_idx = feature_indicies_at_time[0]
        end_idx = feature_indicies_at_time[-1] + 1
        return feature_eet, start_idx, end_idx

In [10]:
for i, rf in enumerate(radar_files):
    if not rf.endswith('.zarr'):
        continue
    radar_dt = dt.strptime(rf, 'KHGX%Y%m%d_%H%M%S_V06_grid.zarr')
    expected_dt = tfm_dts[i]
    if radar_dt != expected_dt:
        raise ValueError(f'Error at index {i}! Expected {expected_dt}, got {radar_dt}')
    rp = radar_top_path + rf
    d, s, e = find_eet_feature(tfm, rp, i)
    feature_eet[s:e] = d

/tmp/ipykernel_1697132/206411139.py:11: RuntimeWarning: All-NaN slice encountered
  feature_eet[j] = np.nanmax(this_eet[this_seg_mask == feat_to_find])
/tmp/ipykernel_1697132/206411139.py:11: RuntimeWarning: All-NaN slice encountered
  feature_eet[j] = np.nanmax(this_eet[this_seg_mask == feat_to_find])
/tmp/ipykernel_1697132/206411139.py:11: RuntimeWarning: All-NaN slice encountered
  feature_eet[j] = np.nanmax(this_eet[this_seg_mask == feat_to_find])
/tmp/ipykernel_1697132/206411139.py:11: RuntimeWarning: All-NaN slice encountered
  feature_eet[j] = np.nanmax(this_eet[this_seg_mask == feat_to_find])


ValueError: Error at index 4! Expected 2022-06-17 00:15:38, got 2022-06-17 00:19:14

In [ ]:
feature_eet

In [ ]:
np.any(tfm.sel(feature=3).segmentation_mask.data == 3)

In [ ]:
radar = xr.open_dataset(rp, engine='zarr')
radar

In [ ]:
from matplotlib import pyplot as plt
import cmweather

fig = plt.figure()
ax = fig.gca()
ax.pcolormesh(radar.x, radar.y, radar.reflectivity.isel(time=0).max(dim='z').data, vmin=-10, vmax=80, cmap='ChaseSpectral')

In [ ]:
fig = plt.figure()
ax = fig.gca()
ax.pcolormesh(radar.x, radar.y, radar.eet_sam.isel(time=0).data)

In [ ]:
tfmt = tfm.isel(time=i)
seg_array = tfmt.segmentation_mask.data.copy().astype(float)
seg_array[seg_array == 0] = np.nan
fig = plt.figure()
ax = fig.gca()
ax.pcolormesh(tfmt.x, tfmt.y, seg_array)

In [ ]:
tfm_new = tfm.assign(
    feature_eet = (('feature'), feature_eet)
)
tfm_new.feature_eet.plot.hist(bins=100)

In [ ]:
dl_res = pd.read_csv('DL_RES.csv')
dl_res['tobac_idx'].values

In [12]:
tfma_path = f'/Volumes/LtgSSD/tobac_saves/tobac_Save_{date_i_want.strftime("%Y%m%d")}/Track_features_merges_augmented.zarr'
tfma = xr.open_dataset(tfma_path, engine='zarr')
tfma['min_L2-MCMIPC'].data

array([295.03062395, 220.94861004, 228.55620089, ..., 237.41712456,
       235.49755321, 241.52625869])

In [13]:
tfma.close()

In [4]:
dataset_time = dt(2022, 7, 12)
path_to_radar = path.join('/Volumes', 'LtgSSD', 'nexrad_l2', dataset_time.strftime('%Y%m%d'))
radar_files = [path.join(path_to_radar, f) for f in sorted(listdir(path_to_radar)) if f.startswith('TIAH')]
radar_times = [dt.strptime(path.basename(f)[4:-4], '%Y%m%d_%H%M%S') for f in radar_files]

In [5]:
radar_times

[]